In [1]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [2]:
# DOWNLOAD THE VGG16 MODEL
import torchvision.models as models
vgg16=models.vgg16(pretrained=True)

In [3]:
dataroot = "./denoised/"
# DATASET OF 20K IMAGES FOR TRAINING

In [4]:
#PARAMETERS FOR LOADING DATASET

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 6
# Spatial size of training images. All images will be resized to this.
image_size = 64
image_size_224 = 224

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 699

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 10000

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [5]:
# Learning rate for optimizers
lr = 0.0001

# Beta hyperparam for Adam optimizers
beta1 = 0.9
beta2 = 0.999

In [6]:
# Create the dataset, dataset stores 64*64*3 images and 224*224*3 images
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
dataset_224= dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size_224),
                               transforms.CenterCrop(image_size_224),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader for both datasets
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=workers)
dataloader_224 = torch.utils.data.DataLoader(dataset_224, batch_size=batch_size,
                                         shuffle=False, num_workers=workers)
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# # Plot some training images
# real_batch = next(iter(dataloader))
# plt.figure(figsize=(8,8))
# plt.axis("off")
# plt.title("Training Images")
# plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))
# real_batch = next(iter(dataloader_224))
# plt.figure(figsize=(8,8))
# plt.axis("off")
# plt.title("Training Images")
# plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:224], padding=2, normalize=True).cpu(),(1,2,0)))

In [7]:
print("Running on: ",device)

Running on:  cuda:0


In [8]:
#first 14 layers of vgg16
first_14_fun_cuda=lambda x:vgg16.classifier.train(mode=False).to(device)[:2](vgg16.avgpool(vgg16.features.train(mode=False).to(device)(x)).to(device).detach().reshape(-1,25088))
def phi(image):
  ftr_img=first_14_fun_cuda(image.to(device)).cpu().detach()
  return ftr_img

In [9]:
#TEST CELL
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
fixed_noise = torch.randn(2, 3, 224, 224, device=device)
test=phi(fixed_noise).reshape(-1,4096)
assert test.shape==(2,4096)

In [10]:
#Eps returns the relu3_3 output of vgg16 model
E=lambda x:vgg16.features.train(mode=False).cuda()[:16](x)
def Eps(image):
  ftr_img=E(image.cuda()).cpu()
  return ftr_img
phi_Eps_16=lambda x:vgg16.classifier.train(mode=False).to(device)[:2](vgg16.avgpool(vgg16.features[16:].train(mode=False).to(device)(x)).to(device).detach().reshape(-1,25088))
#phi_Eps applies on the Eps output of an image to generate the phi output
def phi_Eps(image):
  ftr_img=phi_Eps_16(image.to(device)).cpu().detach()
  return ftr_img

In [11]:
#TEST CELL
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
fixed_noise = torch.randn(2, 3, 224, 224, device=device)
test=phi_Eps(Eps(fixed_noise))
test2=phi(fixed_noise)
assert np.allclose(test.numpy(),test2.numpy())

In [12]:
#returns a matrix which can be used to apply PCA on PyTorch Tensors
def PCA_fit(X, k=2):
 # preprocess the data
 X_mean = torch.mean(X,0)
 X = X - X_mean.expand_as(X)
 # svd
 U,S,V = torch.svd(torch.t(X))
 return U[:,:k]
# Takes as input a tensor and pca_mat to generate the transformed tensor
def PCA_transform(X,U):
  return torch.mm(X,U)

In [13]:
### Can be used to generate the pca_mat matrix
# i=0
# first=False
# for batch in dataloader_224:
#   if first==False:
#     first=True
#     phi_x=phi(batch[0]).reshape(-1,4096)
#   else:
#     phi_x=torch.cat((phi_x,phi(batch[0]).reshape(-1,4096)))
#   if i%32==0:
#     print(i)
#   i=i+1
# PATH_PCAMAT='./CHECKPOINT/pcamat.pth'
# pca_mat=PCA_fit(phi_x,k=699)
# torch.save({'pcamat':pca_mat},PATH_PCAMAT)

In [14]:
PATH_PCAMAT='./CHECKPOINT/pcamat.pth'
checkpoint=torch.load(PATH_PCAMAT)
pca_mat=checkpoint['pcamat']

In [15]:
# custom weights initialization called on netG and netD/
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight, 0.0, 0.1)
        # WE USED THIS FOR FASTER CONVERGENCE 
        # ACTUAL PAPER USES: nn.init.normal_(m.weight.data, 1.0, 0.01) 
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight, 0.0, 0.2)
        nn.init.constant_(m.bias, 0)

In [16]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=True),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=True),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [17]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))
# Apply the weights_init function to randomly initialize all weights
netG.apply(weights_init)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(699, 512, kernel_size=(4, 4), stride=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (13): Tanh()
  )
)

In [18]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=True),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [19]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
netD.apply(weights_init)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1))
    (12): Sigmoid()
  )
)

In [20]:
# Initialize Loss functions
criterion = nn.BCELoss()
crit_mse = nn.MSELoss()

# Establish convention for real and fake labels during training
real_label = 0.95 ##
fake_label = 0.01 ##

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, beta2))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, beta2))

In [21]:
real_label = 0.9999 ##
fake_label = 0.0001 ##

In [22]:
#INITIALISATION CODE
random.seed(1221)
netG.apply(weights_init)
netD.apply(weights_init)
# # Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0
LAMBDA_DIS=100
LAMBDA_ADV=100
LAMBDA_FEA=0.01
LAMBDA_STI=0.0001
epoch_ck=0
upsampler = nn.UpsamplingBilinear2d(size=(224, 224))

In [23]:
SV_PATH='./CHECKPOINT/'
LAMBDA_DIS=100
LAMBDA_ADV=100
LAMBDA_FEA=10
LAMBDA_STI=20000

In [24]:
# LOAD WEIGHTS OF PREVIOUS TRAINING SESSION
LD_PATH='./CHECKPOINT/savedmodeldenoisedsupering1067.pth'
checkpoint = torch.load(LD_PATH)
# print(checkpoint)
netG.load_state_dict(checkpoint['modelG_state_dict'])
optimizerG.load_state_dict(checkpoint['optimizerG_state_dict'])
netD.load_state_dict(checkpoint['modelD_state_dict'])
optimizerD.load_state_dict(checkpoint['optimizerD_state_dict'])
epoch_ck = checkpoint['epoch']

In [25]:
# # Training Loop
# saveNumber=0
# print("Starting Training Loop...")
# # For each epoch
# for epoch in range(epoch_ck,num_epochs):
#     # For each batch in the dataloader
#     for i, dataTuple in enumerate(zip(dataloader,dataloader_224), 0):

#         ############################
#         # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
#         ###########################
       
        
#         data=dataTuple[0]
#         data_224=dataTuple[1]
        
#         # Format batch
#         real_cpu = data[0].to(device)
#         b_size = real_cpu.size(0)
#         real_cpu_224 = data_224[0].to(device)
#         b_size = real_cpu_224.size(0)
        
#         # Generate batch of latent vectors
#         with torch.no_grad():
#             eps_real_cpu_224=Eps(real_cpu_224)
#             phi_real_cpu_224=phi_Eps(eps_real_cpu_224)
#             z=PCA_transform(phi_real_cpu_224,pca_mat)
#             z=z.reshape(z.shape[0],z.shape[1],1,1).to(device)
#             noise=z
        
#         netD.zero_grad() # Init gradients to 0
#         ## Train with all-real batch
#         label = torch.full((b_size,), real_label, device=device)
#         output = netD(real_cpu).view(-1) # Forward pass real batch through D
#         errD_real = LAMBDA_DIS*criterion(output, label) # Calculate loss on all-real batch
#         errD_real.backward() # Calculate gradients for D in backward pass
#         D_x = output.mean().item() # For tracking progress

#         ## Train with all-fake batch        
#         fake = netG(noise) # Generate fake image batch with G
#         label.fill_(fake_label) # Classify all fake batch with D
#         output = netD(fake.detach()).view(-1) # Calculate D's loss on the all-fake batch
#         errD_fake = LAMBDA_DIS*criterion(output, label) # Calulate error for this batch
#         errD_fake.backward() # Calculate the gradients for this batch
#         D_G_z1 = output.mean().item() # For tracking progress
        
#         errD = errD_real + errD_fake # Add the gradients from the all-real and all-fake batches
#         optimizerD.step() # Update D

#         ############################
#         # (2) Update G network: maximize log(D(G(z))) + LOSS_FEA + LOSS_STI
#         ###########################
#         netG.zero_grad()
        
#         label.fill_(real_label)  # fake labels are real for generator cost
#         output = netD(fake).view(-1)# Since we just updated D, perform another forward pass of all-fake batch through D
#         errG_a = LAMBDA_ADV * criterion(output, label)# Calculate G's loss based on this output
#         errG_a.backward()# Calculate gradients for G
        
        
#         G_z=netG(noise.detach()) # Generate an image
#         G_z_224=upsampler(G_z)  # Upsample it to pass through vgg16
#         eps_G_z_224=Eps(G_z_224)# Get features of Generator output 
#         errG_fea= LAMBDA_FEA*crit_mse(eps_real_cpu_224,eps_G_z_224)# Get feature loss
#         errG_fea.backward()# Backward pass 
        
#         G_z2=netG(noise.detach()) # Generate an image
#         errG_sti = LAMBDA_STI*crit_mse(real_cpu,G_z2)# Get stimuli loss
#         errG_sti.backward() #Backward Pass

#         errG=errG_a+errG_fea+errG_sti # Total loss 
#         D_G_z2 = output.mean().item() # For tracking progress
        
#         optimizerG.step() # Update parameters of Generator

#         # Output training stats
#         if i % 5  == 0:
#             print('[%d/%d][%d/%d]\tLoss_D: %.4f,%.4f\tLoss_G: %.4f,%.4f,%.4f\tD(x): %.4f  D(G(z)): %.4f,%.4f'
#                   % (epoch, num_epochs, i, len(dataloader),
#                      errD_fake.item(), errD_real.item(), errG_a.item(),errG_fea.item(),errG_sti.item(), D_x, D_G_z1, D_G_z2))
#             G_losses.append(errG.item())
#             D_losses.append(errD.item())
#         # Periodically save weights for retraining
#         if(iters%500==0):
#             epoch_ck=epoch
#             torch.save({
#             'epoch': epoch,
#             'modelG_state_dict': netG.state_dict(),
#             'optimizerG_state_dict': optimizerG.state_dict(),
#             'errG': errG,
#             'modelD_state_dict': netD.state_dict(),
#             'optimizerD_state_dict': optimizerD.state_dict(),
#             'errD': errD,
#             }, SV_PATH+'savedmodeldenoised'+str(epoch)+'.pth')
#             saveNumber += 1
#             print('SAVED')
#         iters += 1

In [ ]:
# Training Loop
saveNumber=0
print("Starting Training Loop...")
# For each epoch
for epoch in range(epoch_ck,num_epochs):
    # For each batch in the dataloader
    for i, dataTuple in enumerate(zip(dataloader,dataloader_224), 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
       
        
        data=dataTuple[0]
        data_224=dataTuple[1]
        
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        real_cpu_224 = data_224[0].to(device)
        b_size = real_cpu_224.size(0)
        
        # Generate batch of latent vectors
        with torch.no_grad():
            eps_real_cpu_224=Eps(real_cpu_224)
            phi_real_cpu_224=phi_Eps(eps_real_cpu_224)
            z=PCA_transform(phi_real_cpu_224,pca_mat)
            z=z.reshape(z.shape[0],z.shape[1],1,1).to(device)
            noise=z
        
        netD.zero_grad() # Init gradients to 0
        ## Train with all-real batch
        label = torch.full((b_size,), real_label, device=device)
        output = netD(real_cpu).view(-1) # Forward pass real batch through D
        errD_real = LAMBDA_DIS*criterion(output, label) # Calculate loss on all-real batch
        errD_real.backward() # Calculate gradients for D in backward pass
        D_x = output.mean().item() # For tracking progress

        ## Train with all-fake batch        
        fake = netG(noise) # Generate fake image batch with G
        label.fill_(fake_label) # Classify all fake batch with D
        output = netD(fake.detach()).view(-1) # Calculate D's loss on the all-fake batch
        errD_fake = LAMBDA_DIS*criterion(output, label) # Calulate error for this batch
        errD_fake.backward() # Calculate the gradients for this batch
        D_G_z1 = output.mean().item() # For tracking progress
        
        errD = errD_real + errD_fake # Add the gradients from the all-real and all-fake batches
        optimizerD.step() # Update D

        ############################
        # (2) Update G network: maximize log(D(G(z))) + LOSS_FEA + LOSS_STI
        ###########################
        netG.zero_grad()
        
        label.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake).view(-1)# Since we just updated D, perform another forward pass of all-fake batch through D
        errG_a = LAMBDA_ADV * criterion(output, label)# Calculate G's loss based on this output
        errG_a.backward()# Calculate gradients for G
        
        G_z2=netG(noise.detach()) # Generate an image
        errG_sti = LAMBDA_STI*crit_mse(real_cpu,G_z2)# Get stimuli loss
        errG_sti.backward() #Backward Pass

        errG=errG_a+errG_sti # Total loss 
        D_G_z2 = output.mean().item() # For tracking progress
        
        optimizerG.step() # Update parameters of Generator

        # Output training stats
        if i % 5  == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f,%.4f\tLoss_G: %.4f,%.4f\tD(x): %.4f  D(G(z)): %.4f,%.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD_fake.item(), errD_real.item(), errG_a.item(),errG_sti.item(), D_x, D_G_z1, D_G_z2))
            G_losses.append(errG.item())
            D_losses.append(errD.item())
        # Periodically save weights for retraining
        if(iters%1000==0):
            epoch_ck=epoch
            torch.save({
            'epoch': epoch,
            'modelG_state_dict': netG.state_dict(),
            'optimizerG_state_dict': optimizerG.state_dict(),
            'errG': errG,
            'modelD_state_dict': netD.state_dict(),
            'optimizerD_state_dict': optimizerD.state_dict(),
            'errD': errD,
            }, SV_PATH+'savedmodeldenoisedsupering'+str(epoch)+'.pth')
            saveNumber += 1
            print('SAVED')
        iters += 1

Starting Training Loop...
[1067/10000][0/309]	Loss_D: 0.8444,2.7698	Loss_G: 531.0703,357.2418	D(x): 0.9731  D(G(z)): 0.0079,0.0075
SAVED
[1067/10000][5/309]	Loss_D: 2.5016,2.5260	Loss_G: 382.9344,276.2992	D(x): 0.9755  D(G(z)): 0.0243,0.0227
[1067/10000][10/309]	Loss_D: 1.8018,3.7538	Loss_G: 423.3076,302.3023	D(x): 0.9635  D(G(z)): 0.0174,0.0171
[1067/10000][15/309]	Loss_D: 0.7302,7.0211	Loss_G: 510.2597,260.1756	D(x): 0.9326  D(G(z)): 0.0068,0.0074
[1067/10000][20/309]	Loss_D: 0.9767,5.8195	Loss_G: 493.2878,287.5832	D(x): 0.9441  D(G(z)): 0.0092,0.0108
[1067/10000][25/309]	Loss_D: 1.1738,4.7546	Loss_G: 468.3181,268.4562	D(x): 0.9542  D(G(z)): 0.0112,0.0129
[1067/10000][30/309]	Loss_D: 5.1363,1.5610	Loss_G: 304.4213,318.7903	D(x): 0.9850  D(G(z)): 0.0496,0.0501
[1067/10000][35/309]	Loss_D: 9.6311,0.3647	Loss_G: 263.3728,273.5971	D(x): 0.9970  D(G(z)): 0.0909,0.0782
[1067/10000][40/309]	Loss_D: 5.6203,0.2778	Loss_G: 337.2251,337.5583	D(x): 0.9979  D(G(z)): 0.0538,0.0403
[1067/10000][45/

[1069/10000][135/309]	Loss_D: 9.2473,4.6192	Loss_G: 510.2430,415.6958	D(x): 0.9575  D(G(z)): 0.0752,0.0523
[1069/10000][140/309]	Loss_D: 1.4682,0.2659	Loss_G: 653.9780,460.9063	D(x): 0.9981  D(G(z)): 0.0136,0.0125
[1069/10000][145/309]	Loss_D: 0.1191,15.2743	Loss_G: 872.5255,357.7965	D(x): 0.8702  D(G(z)): 0.0003,0.0003
[1069/10000][150/309]	Loss_D: 5.5813,0.3474	Loss_G: 407.3752,434.4562	D(x): 0.9972  D(G(z)): 0.0519,0.0431
[1069/10000][155/309]	Loss_D: 13.9995,0.1929	Loss_G: 342.6081,406.6768	D(x): 0.9989  D(G(z)): 0.1126,0.0770
[1069/10000][160/309]	Loss_D: 0.3394,1183.9851	Loss_G: 3274.8157,3432.1035	D(x): 0.0004  D(G(z)): 0.0000,0.0000
[1069/10000][165/309]	Loss_D: 0.2257,577.2753	Loss_G: 2030.8733,2833.1724	D(x): 0.2707  D(G(z)): 0.0000,0.0000
[1069/10000][170/309]	Loss_D: 0.2285,0.2746	Loss_G: 749.4278,2537.6694	D(x): 0.9981  D(G(z)): 0.0014,0.0090
[1069/10000][175/309]	Loss_D: 75.1637,0.1716	Loss_G: 212.6286,2190.4827	D(x): 1.0000  D(G(z)): 0.2904,0.3055
[1069/10000][180/309]	L

[1071/10000][270/309]	Loss_D: 19.9840,0.5378	Loss_G: 543.1765,3121.7568	D(x): 0.9952  D(G(z)): 0.1311,0.0721
[1071/10000][275/309]	Loss_D: 0.2379,10.3644	Loss_G: 829.8748,3314.1196	D(x): 0.9219  D(G(z)): 0.0016,0.0013
[1071/10000][280/309]	Loss_D: 0.2733,1.3656	Loss_G: 801.6042,3036.4468	D(x): 0.9873  D(G(z)): 0.0019,0.0017
[1071/10000][285/309]	Loss_D: 0.9097,0.8513	Loss_G: 489.1031,2743.7490	D(x): 0.9923  D(G(z)): 0.0086,0.0090
[1071/10000][290/309]	Loss_D: 2.2060,24.9983	Loss_G: 447.4806,3524.1970	D(x): 0.8697  D(G(z)): 0.0212,0.0240
[1071/10000][295/309]	Loss_D: 15.2730,0.4378	Loss_G: 353.8033,2586.2334	D(x): 0.9965  D(G(z)): 0.1356,0.0987
[1071/10000][300/309]	Loss_D: 4.5919,1.0124	Loss_G: 487.2574,3105.5933	D(x): 0.9905  D(G(z)): 0.0432,0.0143
[1071/10000][305/309]	Loss_D: 0.1977,5.2482	Loss_G: 1007.9469,3937.0630	D(x): 0.9503  D(G(z)): 0.0010,0.0008
[1072/10000][0/309]	Loss_D: 24.3235,0.2764	Loss_G: 423.5345,955.3280	D(x): 1.0000  D(G(z)): 0.1553,0.1074
[1072/10000][5/309]	Loss_

[1074/10000][100/309]	Loss_D: 0.3011,0.5872	Loss_G: 742.3367,380.8160	D(x): 0.9948  D(G(z)): 0.0023,0.0021
[1074/10000][105/309]	Loss_D: 0.4375,0.3068	Loss_G: 616.2173,323.4764	D(x): 0.9976  D(G(z)): 0.0038,0.0033
[1074/10000][110/309]	Loss_D: 14.9391,1.1755	Loss_G: 342.5788,310.9194	D(x): 0.9891  D(G(z)): 0.1165,0.0765
[1074/10000][115/309]	Loss_D: 0.3314,0.7608	Loss_G: 625.2737,357.2720	D(x): 0.9930  D(G(z)): 0.0027,0.0030
[1074/10000][120/309]	Loss_D: 15.6405,1.9173	Loss_G: 395.2388,296.1050	D(x): 0.9819  D(G(z)): 0.1153,0.0771
[1074/10000][125/309]	Loss_D: 1.2566,1.7778	Loss_G: 569.8681,349.6132	D(x): 0.9829  D(G(z)): 0.0118,0.0111
[1074/10000][130/309]	Loss_D: 2.3458,2.8246	Loss_G: 424.1481,347.0127	D(x): 0.9737  D(G(z)): 0.0227,0.0178
[1074/10000][135/309]	Loss_D: 2.5494,1.1642	Loss_G: 446.3108,384.2577	D(x): 0.9890  D(G(z)): 0.0242,0.0220
[1074/10000][140/309]	Loss_D: 4.7340,0.1760	Loss_G: 473.7074,432.8482	D(x): 0.9990  D(G(z)): 0.0439,0.0373
[1074/10000][145/309]	Loss_D: 0.561

[1076/10000][235/309]	Loss_D: 2.8730,0.3412	Loss_G: 527.9310,1937.5442	D(x): 0.9973  D(G(z)): 0.0275,0.0171
[1076/10000][240/309]	Loss_D: 0.1197,19.6881	Loss_G: 969.4896,1923.9349	D(x): 0.8327  D(G(z)): 0.0002,0.0002
[1076/10000][245/309]	Loss_D: 1.6543,0.6688	Loss_G: 562.4559,2440.8064	D(x): 0.9940  D(G(z)): 0.0157,0.0168
[1076/10000][250/309]	Loss_D: 0.3077,0.1423	Loss_G: 755.7081,2123.3445	D(x): 0.9994  D(G(z)): 0.0023,0.0024
[1076/10000][255/309]	Loss_D: 0.5902,1.0101	Loss_G: 725.2817,2223.4089	D(x): 0.9908  D(G(z)): 0.0052,0.0028
[1076/10000][260/309]	Loss_D: 0.4500,64.9200	Loss_G: 729.5649,1583.7582	D(x): 0.5825  D(G(z)): 0.0037,0.0048
[1076/10000][265/309]	Loss_D: 0.6868,43.4428	Loss_G: 668.8207,2590.3838	D(x): 0.7233  D(G(z)): 0.0061,0.0075
[1076/10000][270/309]	Loss_D: 13.6828,0.1425	Loss_G: 547.3123,2512.8420	D(x): 0.9993  D(G(z)): 0.1106,0.0812
[1076/10000][275/309]	Loss_D: 0.5317,0.2516	Loss_G: 665.9225,2518.1050	D(x): 0.9984  D(G(z)): 0.0046,0.0038
[1076/10000][280/309]	Lo

...

[1148/10000][245/309]	Loss_D: 2.1285,1.2124	Loss_G: 531.3147,1089.1564	D(x): 0.9885  D(G(z)): 0.0205,0.0087
[1148/10000][250/309]	Loss_D: 0.1171,69.5878	Loss_G: 871.6544,1104.0417	D(x): 0.5326  D(G(z)): 0.0003,0.0003
[1148/10000][255/309]	Loss_D: 0.2510,32.7442	Loss_G: 542.9879,1061.2186	D(x): 0.7290  D(G(z)): 0.0018,0.0058
[1148/10000][260/309]	Loss_D: 5.1407,7.1324	Loss_G: 283.5147,902.5615	D(x): 0.9330  D(G(z)): 0.0488,0.0909
[1148/10000][265/309]	Loss_D: 8.3766,1.0084	Loss_G: 297.3170,1085.6005	D(x): 0.9905  D(G(z)): 0.0776,0.0782
[1148/10000][270/309]	Loss_D: 6.2656,1.8780	Loss_G: 357.0282,1025.2808	D(x): 0.9820  D(G(z)): 0.0582,0.0394
[1148/10000][275/309]	Loss_D: 6.8249,10.3723	Loss_G: 335.8582,1112.8787	D(x): 0.9081  D(G(z)): 0.0651,0.0456
[1148/10000][280/309]	Loss_D: 3.4635,9.3974	Loss_G: 450.2930,1047.2236	D(x): 0.9137  D(G(z)): 0.0320,0.0267
[1148/10000][285/309]	Loss_D: 3.1561,3.5112	Loss_G: 381.6795,944.5569	D(x): 0.9662  D(G(z)): 0.0303,0.0311
[1148/10000][290/309]	Loss_

[1151/10000][75/309]	Loss_D: 7.5337,3.2774	Loss_G: 340.1607,347.8604	D(x): 0.9696  D(G(z)): 0.0704,0.0504
[1151/10000][80/309]	Loss_D: 0.7861,4.4704	Loss_G: 522.1920,339.0958	D(x): 0.9580  D(G(z)): 0.0073,0.0066
[1151/10000][85/309]	Loss_D: 1.9833,1.9708	Loss_G: 466.2841,375.2550	D(x): 0.9809  D(G(z)): 0.0190,0.0170
[1151/10000][90/309]	Loss_D: 0.6476,8.1252	Loss_G: 534.8961,398.7605	D(x): 0.9228  D(G(z)): 0.0059,0.0068
[1151/10000][95/309]	Loss_D: 1.5012,4.3079	Loss_G: 463.3564,340.7809	D(x): 0.9587  D(G(z)): 0.0143,0.0171
[1151/10000][100/309]	Loss_D: 3.1281,1.4729	Loss_G: 350.7048,324.0849	D(x): 0.9858  D(G(z)): 0.0304,0.0316
[1151/10000][105/309]	Loss_D: 33.8195,0.4486	Loss_G: 254.9943,297.7181	D(x): 0.9962  D(G(z)): 0.2247,0.1157
[1151/10000][110/309]	Loss_D: 18.2922,0.8407	Loss_G: 261.3976,317.3582	D(x): 0.9922  D(G(z)): 0.1626,0.0793
[1151/10000][115/309]	Loss_D: 0.4460,9.2634	Loss_G: 613.9719,318.9995	D(x): 0.9136  D(G(z)): 0.0039,0.0033
[1151/10000][120/309]	Loss_D: 0.8420,48.

[1153/10000][215/309]	Loss_D: 0.6680,39.8440	Loss_G: 772.3610,847.7565	D(x): 0.7808  D(G(z)): 0.0059,0.0039
[1153/10000][220/309]	Loss_D: 0.1072,158.6048	Loss_G: 790.0065,902.3740	D(x): 0.3884  D(G(z)): 0.0002,0.0005
[1153/10000][225/309]	Loss_D: 1.4508,4.6056	Loss_G: 367.5226,982.8640	D(x): 0.9568  D(G(z)): 0.0138,0.0569
[1153/10000][230/309]	Loss_D: 20.7409,0.5942	Loss_G: 256.3349,813.5206	D(x): 0.9948  D(G(z)): 0.1713,0.1397
[1153/10000][235/309]	Loss_D: 86.2843,0.1571	Loss_G: 274.9137,835.2489	D(x): 0.9992  D(G(z)): 0.5016,0.1413
[1153/10000][240/309]	Loss_D: 1.6375,8.5305	Loss_G: 524.5573,864.4763	D(x): 0.9235  D(G(z)): 0.0157,0.0093
[1153/10000][245/309]	Loss_D: 1.3662,61.4539	Loss_G: 491.1880,1099.3341	D(x): 0.6124  D(G(z)): 0.0129,0.0173
[1153/10000][250/309]	Loss_D: 24.1966,1.0476	Loss_G: 242.1642,1097.2057	D(x): 0.9902  D(G(z)): 0.1909,0.1593
[1153/10000][255/309]	Loss_D: 22.0615,1.0812	Loss_G: 312.2908,1060.6056	D(x): 0.9899  D(G(z)): 0.1733,0.0893
[1153/10000][260/309]	Loss

[1154/10000][285/309]	Loss_D: 210.7682,0.1399	Loss_G: 98.5722,949.3057	D(x): 0.9994  D(G(z)): 0.7954,0.4339
[1154/10000][290/309]	Loss_D: 1.3224,1.3491	Loss_G: 611.7721,1083.5984	D(x): 0.9872  D(G(z)): 0.0125,0.0053
[1154/10000][295/309]	Loss_D: 0.1437,111.5582	Loss_G: 867.3444,826.3326	D(x): 0.4749  D(G(z)): 0.0005,0.0006
[1154/10000][300/309]	Loss_D: 0.1575,99.9753	Loss_G: 721.7295,1072.8049	D(x): 0.5389  D(G(z)): 0.0007,0.0022
[1154/10000][305/309]	Loss_D: 17.1162,0.7076	Loss_G: 173.6935,1111.3687	D(x): 0.9935  D(G(z)): 0.1389,0.2413
[1155/10000][0/309]	Loss_D: 281.1388,0.1722	Loss_G: 65.2643,438.3846	D(x): 0.9991  D(G(z)): 0.7891,0.6456
[1155/10000][5/309]	Loss_D: 4.3453,2.0596	Loss_G: 442.4897,372.6806	D(x): 0.9802  D(G(z)): 0.0422,0.0122
[1155/10000][10/309]	Loss_D: 0.2920,114.6413	Loss_G: 727.8553,458.8353	D(x): 0.4624  D(G(z)): 0.0022,0.0020
[1155/10000][15/309]	Loss_D: 0.6866,73.0198	Loss_G: 591.0498,372.8433	D(x): 0.6364  D(G(z)): 0.0061,0.0120
[1155/10000][20/309]	Loss_D: 30

[1157/10000][115/309]	Loss_D: 0.5654,19.7126	Loss_G: 542.8099,336.6057	D(x): 0.8264  D(G(z)): 0.0050,0.0091
[1157/10000][120/309]	Loss_D: 13.9989,3.7404	Loss_G: 255.4945,261.3557	D(x): 0.9641  D(G(z)): 0.1218,0.1375
[1157/10000][125/309]	Loss_D: 64.0998,0.7023	Loss_G: 155.9921,314.6515	D(x): 0.9936  D(G(z)): 0.4151,0.2980
[1157/10000][130/309]	Loss_D: 17.9349,0.6283	Loss_G: 271.6691,333.5442	D(x): 0.9943  D(G(z)): 0.1526,0.0826
[1157/10000][135/309]	Loss_D: 7.3924,6.0246	Loss_G: 330.9004,409.0356	D(x): 0.9429  D(G(z)): 0.0703,0.0454
[1157/10000][140/309]	Loss_D: 5.0290,3.1174	Loss_G: 377.7719,435.2271	D(x): 0.9703  D(G(z)): 0.0480,0.0377
[1157/10000][145/309]	Loss_D: 0.5323,48.0334	Loss_G: 691.2293,339.8601	D(x): 0.6523  D(G(z)): 0.0045,0.0059
[1157/10000][150/309]	Loss_D: 0.4193,1.9107	Loss_G: 546.9225,395.3244	D(x): 0.9819  D(G(z)): 0.0036,0.0063
[1157/10000][155/309]	Loss_D: 5.9219,0.2327	Loss_G: 352.7409,386.3860	D(x): 0.9984  D(G(z)): 0.0566,0.0657
[1157/10000][160/309]	Loss_D: 8.

In [ ]:
# EVALUATION
# LOAD TEST DATASET
# torch.save({
#             'epoch': epoch,
#             'modelG_state_dict': netG.state_dict(),
#             'optimizerG_state_dict': optimizerG.state_dict(),
#             'errG': errG,
#             'modelD_state_dict': netD.state_dict(),
#             'optimizerD_state_dict': optimizerD.state_dict(),
#             'errD': errD,
#             }, SV_PATH+'savedmodel'+str(epoch)+'.pth')
# print('SAVED')
dataset = dset.ImageFolder(root='./cropped_test/',
                           transform=transforms.Compose([
                               transforms.Resize(image_size_224),
                               transforms.CenterCrop(image_size_224),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
dataloader_test = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=workers)

In [ ]:
#EVALUATION METRICS
import kornia
def struct_similiarity(x,y):
    ssim=kornia.losses.SSIM(3)
    loss=torch.mean(ssim(x,y))
    return loss
def pearson_coefficient(x,y):
    vx = x - torch.mean(x)
    vy = y - torch.mean(y)
    cost = torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)))
    return cost
featureExtractor=lambda x:vgg16.classifier[:5].train(mode=False).to(device)[:2](vgg16.avgpool(vgg16.features.train(mode=False).to(device)(x)).to(device).detach().reshape(-1,25088))
def feature_similarity(x,y):
    f1=featureExtractor(x)
    f2=featureExtractor(y)
    euclidean = nn.PairwiseDistance(p=2)
    cost = euclidean(f1,f2).mean()
    return cost

In [ ]:
structure=np.array([])
pearson=np.array([])
feature=np.array([])

for i, data in enumerate(dataloader_test, 0):
    batch=data[0].to(device)
    with torch.no_grad():
        z=PCA_transform(phi(batch),pca_mat)
        z=z.reshape(z.shape[0],z.shape[1],1,1).to(device)
        real=batch
        fake =upsampler(netG(z).detach().to(device))
    ssim=struct_similiarity(real,fake).cpu().detach().numpy()
    corr=pearson_coefficient(real,fake).cpu().detach().numpy()
    fsim=feature_similarity(real,fake).cpu().detach().numpy()
    
    structure=np.append(structure,ssim)
    pearson=np.append(pearson,corr)
    feature=np.append(feature,fsim)
    if i>10:
        break

s_mean=np.mean(structure)
p_mean=np.mean(pearson)
f_mean=np.mean(feature)

s_sd=np.std(structure)
p_sd=np.std(pearson)
f_sd=np.std(feature)

In [ ]:
print('Structural Similarity: ',s_mean,' +- ',s_sd)
print('Pearson Correlation: ',p_mean,' +- ',p_sd)
print('Feature Similarity: ',f_mean,' +- ',f_sd)

In [ ]:
# VISUALISE SOME OF THE OUTPUTS
real_batch = next(iter(dataloader_test))
with torch.no_grad():
    k=0
    # VARY K to display different images
    z=PCA_transform(phi(real_batch[0]),pca_mat)
    z=z[k].reshape(1,z.shape[1],1,1).to(device)
    real=real_batch[0][k]
    fake = netG(z).detach().to(device)
real_img=vutils.make_grid(real, padding=2, normalize=True)
fake_img=vutils.make_grid(fake, padding=2, normalize=True)
plt.figure(0)
plt.imshow(real_img.cpu().permute(1, 2, 0))
plt.figure(1)
plt.imshow(fake_img.cpu().permute(1, 2, 0))